In [5]:
file = 'imf-eiu/indicators/CAB (%GDP).xlsx'

In [10]:
import pandas as pd
import numpy as np

In [89]:
# read all rows except the last called IDB thus skip 3 from footer
df = pd.read_excel(file, header=1, skiprows=0, skip_footer=3, na_values=['..','--'], index_col=None)

In [90]:
regions = df.index.tolist()

In [91]:
df.index = list(range(0,57))

In [92]:
df.insert(0,'Regions', regions)

In [13]:
file2 = 'imf-eiu/indicators/CAB (%GDP)-G.xlsx'

In [14]:
df2 = pd.read_excel(file2, header=1, skiprows=0, skip_footer=2, na_values=['..','--'])

In [16]:
df2.dropna(how='all', inplace=True)
df2.drop('Unnamed: 1', axis=1, inplace=True)

In [97]:
df_all = pd.concat([df,df2])

In [211]:
def country_html2(filename, title, xl):
    import numpy as np
    file = 'imf-eiu/indicators/{}.xlsx'.format(filename) #for countries
    df = pd.read_excel(file, header=1, skiprows=0, skip_footer=3, na_values=['..','--'], index_col=None)
    regions = df.index.tolist() # create a new col with country names
    df.index = list(range(0,57)) # create index as seq numbers
    df.insert(0,'Regions', regions) #now we have a col called regions
    
    file2 = 'imf-eiu/indicators/{}-G.xlsx'.format(filename) #for aggregates
    df2 = pd.read_excel(file2, header=1, skiprows=0, skip_footer=2, na_values=['..','--'])
    df2.dropna(how='all', inplace=True)
    df2.drop('Unnamed: 1', axis=1, inplace=True)
    df = pd.concat([df,df2])
    head = '''
     <h1>{} 
                <a href='indicators/xl/{}.xlsx'>
                  <i class="fa fa-file-excel-o" aria-hidden="true" style="float: right"></i>
                  </a>
                  <a href='indicators/pdf/{}.pdf'>
                  <i class="fa fa-file-pdf-o" aria-hidden="true" style="float: right">&nbsp;&nbsp;&nbsp;</i>
                  </a>
                  </h1>
                <div class="table-responsive">
                  <table class="table table-striped">
               <thead>
                <tr>
                  <th></th>
                  <th>1990</th>
                  <th>2022</th>
                  <th></th>
                </tr>
               </thead>
               <tbody>

    '''.format(title,xl, xl)
    foot = '''
                </tbody>
              </table>
            </div> 
            <script src="js/line.js"></script>   
        '''
    insert=''
    for index, row in df.iterrows():
        ll = row[1:].tolist() 
        country = row[0]
        npll = np.array(ll)
        loc = np.where(np.isnan(npll))[0]
        if loc.size == 0: #all data years available
            insert = insert + "<tr><th scope='row'>{}</th><td>{:.1f}</td><td>{:.1f}</td><td><canvas id='BHR' class='sparkline' data-chart_values=[{}]></canvas></td></tr>".format(country,row[1],row[33],','.join(str(e) for e in npll.tolist()))
        elif loc.size == 33: #no data available
            insert = insert + "<tr><th scope='row'>{}</th><td>..</td><td>..</td><td><canvas id='BHR'></canvas></td></tr>".format(country)
        else: # some data available
            start = loc[0]
            end = loc[len(loc)-1]
            if start==0:
                focus = row[end+2]
                mydata = ll[end+1:]
                mydata = np.array(mydata)
                insert = insert + "<tr><th scope='row'>{}</th><td>{:.1f}<sup>{}</sup></td><td>{:.1f}</td><td><canvas id='BHR' class='sparkline' data-chart_values=[{}]></canvas></td></tr>".format(row[0],focus,1990+end+1,row[33],','.join(str(e) for e in mydata.tolist()))           
            elif end==32:
                focus = row[start]
                mydata = ll[0:start]
                mydata = np.array(mydata)
                insert = insert + "<tr><th scope='row'>{}</th><td>{:.1f}</td><td>{:.1f}<sup>{}</sup></td><td><canvas id='BHR' class='sparkline' data-chart_values=[{}]></canvas></td></tr>".format(row[0],row[1],focus,1990+start-1,','.join(str(e) for e in mydata.tolist()))           
    final = head + insert + foot
    Html_file= open("{}.html".format(xl),"w")
    Html_file.write(final)
    Html_file.close()

In [212]:
country_html2('RealGDPGrowth', 'Real GDP Growth (% Change)','GDP')

In [214]:
country_html2('Inflation','Inflation, average consumer prices (% Change)', 'inflation')

In [207]:
head = '''
     <h1>{} 
                <a href='indicators/xl/{}.xlsx'>
                  <i class="fa fa-file-excel-o" aria-hidden="true" style="float: right"></i>
                  </a>
                  <a href='indicators/pdf/{}.pdf'>
                  <i class="fa fa-file-pdf-o" aria-hidden="true" style="float: right">&nbsp;&nbsp;&nbsp;</i>
                  </a>
                  </h1>
                <div class="table-responsive">
                  <table class="table table-striped">
               <thead>
                <tr>
                  <th></th>
                  <th>1990</th>
                  <th>2022</th>
                  <th></th>
                </tr>
               </thead>
               <tbody>

    '''.format('cab name file title','cab', 'cab')
foot = '''
                </tbody>
              </table>
            </div> 
            <script src="js/line.js"></script>   
        '''
insert=''
for index, row in df_all.iterrows():
    ll = row[1:].tolist() 
    country = row[0]
    npll = np.array(ll)
    loc = np.where(np.isnan(npll))[0]
    if loc.size == 0: #all data years available
        insert = insert + "<tr><th scope='row'>{}</th><td>{:.1f}</td><td>{:.1f}</td><td><canvas id='BHR' class='sparkline' data-chart_values=[{}]></canvas></td></tr>".format(country,row[1],row[33],','.join(str(e) for e in npll.tolist()))
    elif loc.size == 33: #no data available
        insert = insert + "<tr><th scope='row'>{}</th><td>..</td><td>..</td><td><canvas id='BHR'></canvas></td></tr>".format(country)
    else: # some data available
        start = loc[0]
        end = loc[len(loc)-1]
        if start==0:
            focus = row[end+2]
            mydata = ll[end+1:]
            mydata = np.array(mydata)
            insert = insert + "<tr><th scope='row'>{}</th><td>{:.1f}<sup>{}</sup></td><td>{:.1f}</td><td><canvas id='BHR' class='sparkline' data-chart_values=[{}]></canvas></td></tr>".format(row[0],focus,1990+end+1,row[33],','.join(str(e) for e in mydata.tolist()))           
        elif end==32:
            focus = row[start]
            mydata = ll[0:start]
            mydata = np.array(mydata)
            insert = insert + "<tr><th scope='row'>{}</th><td>{:.1f}</td><td>{:.1f}<sup>{}</sup></td><td><canvas id='BHR' class='sparkline' data-chart_values=[{}]></canvas></td></tr>".format(row[0],row[1],focus,1990+start-1,','.join(str(e) for e in mydata.tolist()))           
final = head + insert + foot

In [204]:
final

'\n     <h1>cab name file title \n                <a href=\'indicators/xl/cab.xlsx\'>\n                  <i class="fa fa-file-excel-o" aria-hidden="true" style="float: right"></i>\n                  </a>\n                  <a href=\'indicators/pdf/cab.pdf\'>\n                  <i class="fa fa-file-pdf-o" aria-hidden="true" style="float: right">&nbsp;&nbsp;&nbsp;</i>\n                  </a>\n                  </h1>\n                <div class="table-responsive">\n                  <table class="table table-striped">\n               <thead>\n                <tr>\n                  <th></th>\n                  <th>1990</th>\n                  <th>2022</th>\n                  <th></th>\n                </tr>\n               </thead>\n               <tbody>\n\n    <tr><th scope=\'row\'>Afghanistan</th><td>34.1<sup>2002</sup></td><td>-3.8</td><td><canvas id=\'BHR\' class=\'sparkline\' data-chart_values=[34.118,29.917,37.517,30.655,21.598,63.947,33.769,41.532,29.224,26.703,10.828,0.333,5.717,